# Louvain Community Detection


In this notebook, we will use cuGraph to identify the cluster in a test graph using the Louvain algorithm  

Notebook Credits
* Original Authors: Bradley Rees and James Wyles
* Last Edit: 08/13/2019

RAPIDS Versions: 0.9.0

Test Hardware
* GV100 32G, CUDA 10.0



## Introduction

The Louvain method of community detection is a greedy hierarchical clustering algorithm which seeks to optimize modularity as it progresses. Louvain starts with each vertex in its own clusters and iteratively merges groups using graph contraction.  

For a detailed description of the algorithm see: https://en.wikipedia.org/wiki/Louvain_Modularity

It takes as input a cugraph.Graph object and returns as output a 
cudf.Dataframe object with the id and assigned partition for each 
vertex as well as the final modularity score


To compute the Louvain cluster in cuGraph use: <br>

**nvLouvain(G)**
* __G__: A cugraph.Graph object

Returns:

* tupal __lovain dataframe__ and __modularity__


* __louvain__: cudf.DataFrame with two names columns:
    * louvain["vertex"]: The vertex id.
    * louvain["partition"]: The assigned partition.
    
* __modularity__ : the overall modularity of the graph

All vertices with the same partition ID are in the same cluster


## cuGraph Notice 
The current version of cuGraph has some limitations:

* Vertex IDs need to be 32-bit integers.
* Vertex IDs are expected to be contiguous integers starting from 0.

cuGraph provides the renumber function to mitigate this problem. Input vertex IDs for the renumber function can be either 32-bit or 64-bit integers, can be non-contiguous, and can start from an arbitrary number. The renumber function maps the provided input vertex IDs to 32-bit contiguous integers starting from 0. cuGraph still requires the renumbered vertex IDs to be representable in 32-bit integers. These limitations are being addressed and will be fixed soon.

## References

* Blondel, V. D., Guillaume, J.-L., Lambiotte, R., and Lefebvre, E. Fast unfolding of communities in large networks. Journal of statistical mechanics: theory and experiment 2008, 10 (2008), P10008.


### Test Data
We will be using the Zachary Karate club dataset 
*W. W. Zachary, An information flow model for conflict and fission in small groups, Journal of
Anthropological Research 33, 452-473 (1977).*


![Karate Club](./img/zachary_black_lines.png)


### Prep

In [ ]:
# Import needed libraries
import cugraph
import cudf
import numpy as np
from collections import OrderedDict

## Read data using cuDF

In [ ]:
# Test file    
datafile='./data//karate-data.csv'

# Read the data file
cols = ["src", "dst"]

dtypes = OrderedDict([
        ("src", "int32"), 
        ("dst", "int32")
        ])

gdf = cudf.read_csv(datafile, names=cols, delimiter='\t', dtype=list(dtypes.values()) )

In [ ]:
# Louvain is dependent on vertex ID starting at zero
gdf["src_0"] = gdf["src"] - 1
gdf["dst_0"] = gdf["dst"] - 1

In [ ]:
# The algorithm also requires that there are vertex weights.  Just use 1.0 
gdf["data"] = 1.0

In [ ]:
# just for fun, let's look at the data types in the dataframe
gdf.dtypes

In [ ]:
# create a Graph 
G = cugraph.Graph()
G.add_edge_list(gdf["src_0"], gdf["dst_0"], gdf["data"])

In [ ]:
# Call Louvain on the graph
df, mod = cugraph.louvain(G) 

In [ ]:
# Print the modularity score
print('Modularity was {}'.format(mod))
print()

In [ ]:
df.dtypes

In [ ]:
# How many partitions where found
part_ids = df["partition"].unique()

In [ ]:
print(str(len(part_ids)) + " partition detected")

In [ ]:
for p in range(len(part_ids)):
    part = []
    for i in range(len(df)):
        if (df['partition'][i] == p):
            part.append(df['vertex'][i] +1)
    print("Partition " + str(p) + ":")
    print(part)


___
Copyright (c) 2019, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___